# Transaction module
* 進行交易
* 定存利息參考：
    * https://www.edh.tw/article/23183/2

In [10]:
import numpy as np 
import pandas as pd
from helper_func import init_account, init_rate, get_one_stock, get_stocks
from profit_loss import profit_loss_analysis

## Initial Transaction info

In [ ]:
def init_transc(strategy = 1):
    info = None
    if strategy == 1:
        info = {
            'close': 0,
            'pre_close': 0,
            'mean': 0,
            'stop_profit': 0,
            'stop_loss': 0,
            'have_day' : 0
        }
    elif strategy == 2:
        info = {
            'M_close': 0,
            '10M_SMA': 0,
        }
    return info

## Transaction Buy & Sell

In [13]:
def buy(investor, price, rate, FD_account=None):
    if FD_account:
        capital = FD_account['principle']
    else:
        capital = investor['capital']
    
    b_price = investor['per_trans'] * (price + rate['slippage'])
    b_price += b_price * rate['handling_fee']
    
    if capital - b_price >= 0:
        investor['cost'] += b_price
        investor['buy_times'] += 1
        investor['shares'] += investor['per_trans']
        if FD_account:
            investor['capital'] = -b_price
        else:
            investor['capital'] -= b_price
    
    return investor, b_price

In [9]:
def sell(investor, price, rate, FD_account=None):
    b_price = investor['shares'] * (price + rate['slippage'])
    b_price *= (1 - rate['handling_fee'])
    b_price *= (1 - rate['trans_tax'])
    investor['earn'] += b_price
    investor['sell_times'] += 1
    investor['shares'] = 0
    if FD_account:
        investor['capital'] = b_price
    else:
        investor['capital'] += b_price
    return investor, b_price

## Update


In [ ]:
# 更新停損停利點
def update_info(info, company, day, investor, strategy=1):
    if strategy == 1:
        info['close'] = company['close'][day]
        info['mean'] = company['middle'][day]

        if investor['shares'] == 0:
            info['have_day'] = 0
        else:
            info['have_day'] += 1
    elif strategy == 2:
        info['M_close'] = company['close'][day]
        info['10M_SMA'] = company['middle'][day]
    return info

In [ ]:
def update_stop_point(info, raise_rate, drop_rate):
    info['stop_profit'] = info['close'] * (1 + raise_rate)
    info['stop_loss'] = info['close'] * (1 - drop_rate)
    return info

In [3]:
# 每個月固定更新
def update_FD_monthly(account, FD_type=1):
    # 存本取息
    if FD_type == 1:
        account['have_month'] += 1
        
        if account['have_month'] >= 9:
            account['year_rate'] = 0.95 / 100
        elif account['have_month'] >= 6:
            account['year_rate'] = 0.835 / 100
        elif account['have_month'] >= 3:
            account['year_rate'] = 0.66 / 100
        else :
            account['year_rate'] = 0.6 / 100

        # Clean up once a year
        if account['have_month'] == 12:
            account['principle'] *= 1 + account['year_rate']
    # 整存整息  
    elif FD_type == 2:
        account['have_month'] += 1
        # [TODO]
    return account

In [ ]:
# 買賣時更新定存
def update_FD(account, inv, FD_type=1):
    # Clean up whenever you deposite or withdraw
    account['principle'] *= 1 + account['year_rate']
    account['principle'] += inv['capital']
    account['have_month'] = 0
    
    return account

## Trade Strategy

In [ ]:
def simple_strategy(info, expiry_period = 60):
    
    if info['close'] > info['mean'] and info['pre_close'] < info['mean']:
        sign = "cross_up"
#     elif info['close'] < info['mean'] and info['pre_close'] > info['mean']:
#         sign = "cross_dwn"
#         sign = "None"
    elif info['close'] >= info['stop_profit']:
        sign = "stop_profit"
    elif info['close'] <= info['stop_loss']:
        sign = "stop_loss"
#     elif info['have_day'] >= expiry_period:
#         sign = "expiry_date"
    else:
        sign = "None"

    return sign, info['close']

In [6]:
def MebFaber_GTAA(info):
    if info['M_close'] > info['10M_SMA']:
        sign = 'buy'
    elif info['M_close'] < info['10M_SMA']:
        sign = 'sell'
    else:
        sign= "None"
    return sign, info['M_close']

In [2]:
# Return sign, price
def trade(info, strategy = 1):
    if strategy == 1:
        return simple_strategy(info)
    elif strategy == 2:
        return MebFaber_GTAA(info)

## Transaction

In [1]:
"""
tickers = ['',''] list of ticker
Do many companys. For Meb Faber's GTAA.
[TODO]: Fixed deposit
""" 
def transactions(tickers, strategy=2, days=300, drop_rate=0.03, raise_rate=0.02, print_earn=False):
    from helper_func import check_if_last_day_of_month, init_FD_account
    
    companys = get_stocks(tickers, days=days)
    # Should get the min total days
    total_days = len(companys[tickers[0]])
    rate = init_rate()
    
    # Initial
    init_capital = 0.0
    investor = {}
    SIG = {}
    info = {}
    record = {}
    for ticker in tickers:
        if strategy == 1:
            investor[ticker] = init_account()
        elif strategy == 2:
            investor[ticker] = init_account(capital=0)
            init_capital += investor[ticker]['init_capital']

        SIG[ticker] = np.zeros(total_days)
        # Initial trading info
        info[ticker] = init_transc()
        record[ticker] = []
        
    if strategy == 1:
        BUY = ["cross_up"]
        SELL = ["cross_dwn", "stop_loss", "stop_profit", "expiry_date"]
    elif strategy == 2:
        BUY = ["buy"]
        SELL = ["sell"]
        print(init_capital)
        FD_account = init_FD_account(principle=init_capital, rate=0.006)

    for day in range(0, total_days):
        for ticker, company in companys.items():

            if strategy == 1:
                info[ticker] = update_info(info[ticker], company, day, investor[ticker], strategy=strategy)
                sign, price = trade(info[ticker], strategy=strategy)
                if sign in BUY and investor[ticker]['shares'] == 0:
                    info[ticker] = update_stop_point(info[ticker], raise_rate, drop_rate)
                    investor[ticker], actuall_price = buy(investor[ticker], price, rate)
                    record[ticker].append([company.index[day], 'BUY', sign, investor[ticker]['capital'],
                                   investor[ticker]['shares'], price, actuall_price, day])
                    SIG[ticker][day] = 1
                elif sign in SELL and investor[ticker]['shares'] != 0:
                    investor[ticker], actuall_price = sell(investor[ticker], price, rate)
                    record[ticker].append([company.index[day], 'SELL', sign, investor[ticker]['capital'],
                                   investor[ticker]['shares'], price, actuall_price, day])
                    SIG[ticker][day] = 2
                info[ticker]['pre_close'] = info[ticker]['close']
            elif strategy == 2:
                date = company.index.date[day]
                if check_if_last_day_of_month(date):
                    info[ticker] = update_info(info[ticker], company, day, investor[ticker], strategy=strategy)
                    sign, price = trade(info[ticker], strategy=strategy)
                    
                    # update FD
                    FD_account = update_FD_monthly(FD_account)

                    if sign in BUY and investor[ticker]['shares'] == 0:
                        
                        investor[ticker], actuall_price = buy(investor[ticker], price, rate, FD_account)
                        FD_account = update_FD(FD_account, inv=investor[ticker])
                        
                        record[ticker].append([company.index[day], 'BUY', sign, investor[ticker]['capital'],
                                       investor[ticker]['shares'], price, actuall_price, day])
                        SIG[ticker][day] = 1
                    elif sign in SELL and investor[ticker]['shares'] != 0:
                        
                        investor[ticker], actuall_price = sell(investor[ticker], price, rate, FD_account)
                        FD_account = update_FD(FD_account, inv=investor[ticker])
                        
                        record[ticker].append([company.index[day], 'SELL', sign, investor[ticker]['capital'],
                                       investor[ticker]['shares'], price, actuall_price, day])
                        SIG[ticker][day] = 2
                        
    # 清空
    total_earn = {}
    for ticker, company in companys.items():
        if investor[ticker]['shares'] != 0:
            _, price = trade(info[ticker], strategy=strategy)
            
            if strategy == 2:
                investor[ticker], actuall_price = sell(investor[ticker], price, rate, FD_account)
                FD_account = update_FD(FD_account, inv=investor[ticker])
            elif strategy == 1:
                investor[ticker], actuall_price = sell(investor[ticker], price, rate)
            
            record[ticker].append([company.index[day], 'SELL', 'clean', investor[ticker]['capital'],
                               investor[ticker]['shares'], info[ticker]['close'], actuall_price, day])
            SIG[ticker][day] = 2
        total_earn[ticker] = investor[ticker]['earn'] - investor[ticker]['cost']
        if print_earn:
            print(f"{ticker}'s Total Earn: {total_earn[ticker]}")
        record[ticker] = pd.DataFrame(record[ticker], columns=['日期', '動作', '指標', '該股總資金', '持有股', '交易每股金額', '實際買賣金額', 'Index'])

    if strategy == 2:
        total_earn['total earn'] = FD_account['principle'] - init_capital
    return record, investor, SIG, total_earn

In [3]:
# return company dataframe, transaction record, investor final
def transaction(company, strategy=1, drop_rate=0.03, raise_rate=0.02, print_earn=False):
    # Investor capital
    investor = init_account()
    SIG = np.zeros(len(company))
    rate = init_rate()

    record = []
    
    # Initial trading info
    info = init_transc()
    
    for day in range(0, len(company)):
        
        info = update_info(info, company, day, investor)
        
        BUY = ["cross_up"]
        SELL = ["cross_dwn", "stop_loss", "stop_profit", "expiry_date"]
        
        sign, price = trade(info)

        if sign in BUY and investor['shares'] == 0:
            info = update_stop_point(info, raise_rate, drop_rate)
            investor, actuall_price = buy(investor, price, rate)
            record.append([company.index[day], 'BUY', sign, investor['capital'],
                           investor['shares'], price, actuall_price, day])
            SIG[day] = 1
        elif sign in SELL and investor['shares'] != 0:
            investor, actuall_price = sell(investor, price, rate)
            record.append([company.index[day], 'SELL', sign, investor['capital'],
                           investor['shares'], price, actuall_price, day])
            SIG[day] = 2
        
        info['pre_close'] = info['close']
        
    # 清空
    if investor['shares'] != 0:
        investor, actuall_price = sell(investor, price, rate)
        record.append([company.index[day], 'SELL', 'clean', investor['capital'],
                           investor['shares'], info['close'], actuall_price, day])
        SIG[day] = 2
    total_earn = investor['earn'] - investor['cost']
    if print_earn:
        print(f"Total Earn: {total_earn}")
    record = pd.DataFrame(record, columns=['日期', '動作', '指標', '總資金', '持有股', '交易每股金額', '實際買賣金額', 'Index'])
    return record, investor, SIG, total_earn

## Get Transaction DF

In [ ]:
# 拿到交易分析 dataFrame 
# ticker: 股票代碼
def get_transaction_DF(ticker , basic_ratio=None):

    # 拿到股票資訊
    company = get_one_stock(ticker)

    # 停利停損
    if basic_ratio is None:
        basic_ratio = np.arange(2,5,1) # 停利為停損的幾倍
    drop_ratios = np.arange(0.01, 0.1, 0.01) # 停損

    # 紀錄
    records = []

    # r 為倍率: 停損的r倍
    for r in basic_ratio:
        for d in drop_ratios:
            _raise_rate = d*r
            # recored: 交易紀錄 investor:交易者資訊 total_earn: 賺多少 
            record, investor, _, total_earn = transaction(company, drop_rate=d, raise_rate=_raise_rate) 
            # WL_rate: 賺賠比 pw: 勝率   
            WL_rate, pw, _ = profit_loss_analysis(record)
            hpr = round(total_earn / investor['init_capital'] * 100 ,2) # 報酬率:%
            # 紀錄
            records.append( [_raise_rate , d , investor['buy_times'] , investor['sell_times'] , WL_rate , pw , hpr ]   )

    df = pd.DataFrame(records ,columns=['停利點','停損點','進場次數','出場次數','W/L','Pw','投資報酬率(%)'])
    return df